In [60]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
import requests

In [39]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [40]:
def summarize_text(text):
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Calculate frequency of each word
    word_frequencies = {}
    for sentence in sentences:
        words = word_tokenize(sentence.lower())
        for word in words:
            if word not in stopwords.words('english'):
                if word not in word_frequencies:
                    word_frequencies[word] = 1
                else:
                    word_frequencies[word] += 1

    # Find weighted frequency of each word
    max_frequency = max(word_frequencies.values())
    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word] / max_frequency)

    # Calculate sentence scores based on word frequencies
    sentence_scores = {}
    for sentence in sentences:
        for word in word_tokenize(sentence.lower()):
            if word in word_frequencies.keys():
                if len(sentence.split(' ')) < 30:  # Limiting sentence length
                    if sentence not in sentence_scores.keys():
                        sentence_scores[sentence] = word_frequencies[word]
                    else:
                        sentence_scores[sentence] += word_frequencies[word]

    # Get the top 3 sentences as the summary
    summary_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:3]
    summary = ' '.join(summary_sentences)
    return summary

In [41]:
def analyze_bias_and_sentiment(summary):
    # Sentiment Analysis
    sid = SentimentIntensityAnalyzer()
    sentiment_score = sid.polarity_scores(summary)
    sentiment = "Positive" if sentiment_score['compound'] >= 0 else "Negative"

    return sentiment, sentiment_score

In [61]:
def extract_article_content(url, token):
    # Construct the API request URL
    api_url = f"https://api.diffbot.com/v3/article"
    params = {
        'token': token,
        'url': url,
    }

    # Send the API request
    response = requests.get(api_url, params=params)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Check if article extraction was successful
        if data.get('objects'):
            article_content = data['objects'][0]['text']
            return article_content
        else:
            return "Failed to extract article content."
    else:
        return "Failed to retrieve data from Diffbot."

# Example URL and API token
url = input("Paste your URL here.")
token = "API Token Here"  # Replace this with your Diffbot API token

# Extract article content
article_content = extract_article_content(url, token)
print(article_content)

Paste your URL here.https://apnews.com/article/boeing-justice-department-737-max-82145b25ed988cd8cae0bce3de79ce9d
WASHINGTON (AP) — Boeing has violated a settlement that allowed the company to avoid criminal prosecution after two deadly crashes involving its 737 Max aircraft more than five years ago, the Justice Department told a federal judge on Tuesday.
It is now up to the Justice Department to decide whether to file charges against Boeing. Prosecutors will tell the court no later than July 7 how they plan to proceed, department said.
New 737 Max jets crashed in 2018 in Indonesia and 2019 in Ethiopia, killing 346 people. Boeing reached a $2.5 billion settlement with the Justice Department in January 2021 to avoid prosecution on a single charge of fraud — misleading federal regulators who approved the plane. Boeing blamed the deception on two relatively low-level employees.
In a letter filed Tuesday in federal court in Texas, Glenn Leon, head of the Justice Department criminal divisio

In [62]:
#Text Input
input_text = article_content

# Summarize the input text
summary = summarize_text(input_text)

# Analyze bias and sentiment of the summary
sentiment, sentiment_score = analyze_bias_and_sentiment(summary)

print("Summary:")
print(summary)
print("\nSentiment:", sentiment)
print("Sentiment Score:", sentiment_score)

Summary:
The settlement included a $243.6 million fine, a $500 million fund for victim compensation, and nearly $1.8 billion to airlines whose Max jets were grounded for nearly two years. “The Government is determining how it will proceed in this matter,” the Justice Department said in the court filing. After secret negotiations, the government agreed not to prosecute Boeing on a charge of defrauding the United States by deceiving regulators about the flight system.

Sentiment: Positive
Sentiment Score: {'neg': 0.056, 'neu': 0.789, 'pos': 0.155, 'compound': 0.7819}
